In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

/home/huizhu/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import sys
sys.path.append('/home/huizhu/bitbucket/')
from Erwin.utils import rdkit_utils, obabel_utils

In [ ]:
dude_labels_path = '/home/huizhu/bitbucket/mol_data/dude/labels.csv'
df = pd.read_csv(dude_labels_path)
df.shape

(1200966, 104)

In [2]:

from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input, merge
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
import keras.backend as K

In [7]:
input_shape = (224, 224, 3)
model = Sequential()

In [10]:
model.add(Convolution2D(16, (1, 1), input_shape=input_shape, padding='same', use_bias=False))

In [11]:
model.input_shape, model.output_shape

((None, 224, 224, 3), (None, 224, 224, 16))